<a href="https://colab.research.google.com/github/VictorGazzinelli/Frameworks-Backend-Python/blob/main/Exemplo_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercicio 2

## 1 - Como organizar modelos em modulos?

Modelos dentro do paradigma de orientação a objetos são representados como classes correspondentes a uma ou mais tabelas de um banco de dados relacional, os POPOs (Plain Old Python Object) se assemelham aos Java Beans e aos POCOs (C#).

É recomendado organizar diferentes tipos de modelos dentro de um diretório "models" na raíz da aplicação. Por sua vez, dentro desse diretório haverá um script \_\_init__.py para agregar modelos visando sua importação por outros modulos sem necessidade de explicitar o namespace/caminho completo até o modelo.

## 2 - Como criar modelos com heranças? De quais tipos?

In [ ]:
from django.db import models

class Departamento(models.Model):
	nome = models.CharField(max_length=50)
	numero_sala = models.IntegerField(default=0)
 
class Telefone(models.Model):
	ddd = models.IntegerField(default=0)
	numero = models.BigIntegerField(default=0)
	temWhatsapp = models.BooleanField()

class Pessoa(models.Model):
	SEXO_CHOICES = [('M','Masculino'),
					('F','Feminino')]

	nome = models.CharField(max_length=50)
	sobrenome = models.CharField(max_length=50,
							null=True)
	idade = models.IntegerField(default=0)
	sexo = models.CharField(max_length=10,
			choices=SEXO_CHOICES,
			default='M')
	dataNascimento = models.DateField(null=True)
	departamento = models.ForeignKey(Departamento,
							on_delete=models.SET_NULL,
							null=True)
	telefones = models.ManyToManyField(Telefone)

	def __str__(self):
		return f"{self.nome} {self.sobrenome}"

class Cliente(Pessoa):
    numero_compras = models.IntegerField(default=0)

class Funcionario(Pessoa):
    salario = models.DecimalField(max_digits=15, decimal_places=2)

## 3 - Como criar Enumeration types e usar como choices?

In [ ]:
from django.db import models

class Pessoa(models.Model):
	SEXO_CHOICES = [('M','Masculino'),
					('F','Feminino')]

	nome = models.CharField(max_length=50)
	sobrenome = models.CharField(max_length=50,
							null=True)
	idade = models.IntegerField(default=0)
	sexo = models.CharField(max_length=10,
			choices=SEXO_CHOICES,
			default='M')

## 4 - No projeto da aula, crie pelo menos quatro queries customizadas. Use as informações em https://docs.djangoproject.com/en/3.0/topics/db/queries/

In [ ]:
from django.db import models

class PessoaManager(models.Manager):
	def obter_pessoa_id(self, idpessoa):
		return self.get(id=idpessoa)

	def obter_todas_pessoas(self):
		return self.all()

	def obter_maiores_18(self):
		pessoas = self.filter(idade__gte=18)
		return pessoas

  def obter_menores_18(self):
		pessoas = self.filter(idade__lt=18)
		return pessoas

  def obter_victor(self):
		pessoas = self.filter(nome="Victor")
		return pessoas

	def create_pessoa(self, nome_completo):
		_nome_completo = nome_completo.split(" ")
		pessoa = self.create(nome=_nome_completo[0],
							 sobrenome=_nome_completo[1])
		return pessoa

## 5 - O que é, para quê serve e como usar um Proxy no modelo?

Ao utilizar herança de várias tabelas, uma nova tabela de banco de dados é criada vinculando a FK da classe base (no nosso exemplo, pessoa) junto aos atributos desejados a superclasse. Geralmente este é o comportamento desejado, porém existem casos em que a "estrutura de dados" da tabela se torna uma "classe" com funções e métodos que definem o seu comportamento ao longo da aplicação, fazendo com que seja necessário a criação do modelo proxy.

A herança do modelo proxy visa criar um proxy para o modelo original. podendo criar, excluir e atualizar instâncias do modelo de proxy e todos os dados serão salvos como se você estivesse usando o modelo original. A principal diferença esta em alterar itens como a ordem do modelo padrão ou o manager padrão no proxy, sem precisar alterar o original, assim como no Design Pattern.

In [ ]:
from django.db import models

class Person(models.Model):
    first_name = models.CharField(max_length=30)
    last_name = models.CharField(max_length=30)

class MyPerson(Person):
    class Meta:
        proxy = True

    def do_something(self):
        # ...
        pass

## 6 - O que faz o método \_\_str__() em uma classe?

Esse método tem como objetivo sobreescrever a serialização do objeto em questão como uma cadeia de caracteres, muito similar ao método .toString em Java, C# e Javascript. Se o mesmo não for sobrescrito, tem como valor padrão o template \[{tipo} object]. Cabe ao programador definir esta função com o valor mais adequado de representação daquele objeto.

In [ ]:
from django.db import models

class Pessoa(models.Model):
	nome = models.CharField(max_length=50)
	sobrenome = models.CharField(max_length=50,
							null=True)
 
 	def __str__(self):
		return f"{self.nome} {self.sobrenome}"

## 7 - Quais são e para que servem as propriedades adicionais dos tipos de relacionamento/mapeamento?

As propriedades adicionais dos tipos de relacionamento/mapeamento indicam a cardinalidade da relação entre os objetos e quais objetos são relacionados (ManyToMany, OneToMany, ForeignKey ...)

No caso da ForeignKey, podem também ser definidos se são null (Cardinalidade 0),
e metodos on_delete que especificam o comportamento a ser executado caso a entidade referenciada por aquela FK Seja excluida. Existem vários métodos, alguns deles são:

models.CASCADE => exclui todas as entidades que referenciam aquela FK,
models.PROTECT => levanta um erro do tipo ProtectedError, protegendo a exclusao executando um rollback no controle de transacao.
models.SET_NULL => transforma todos os filhos a uma FK nula,
models.SET_DEFAULT (...) a uma FK com valor padrao.